In [1]:
# Import our dependencies
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
# import tensorflow as tf
# import seaborn as sns
import matplotlib.pyplot as plt
# from geopy import Nominatim
%matplotlib inline
# %seaborn inline


In [2]:
# ****** DATASET 1 : THE FIRE DATA *******

#  Import and read the charity_data.csv.
import pandas as pd 
fire_df = pd.read_csv("Data/California_Fire_Incidents.csv")
fire_df[fire_df["CountyIds"] == '6,11,17,23']["Name"]

1054    Ranch Fire (Mendocino Complex)
1055    Ranch Fire (Mendocino Complex)
1056    Ranch Fire (Mendocino Complex)
1057    Ranch Fire (Mendocino Complex)
Name: Name, dtype: object

In [3]:
fire_df = fire_df.drop(["Active", 
"AdminUnit", 
"AirTankers",
"CanonicalUrl", 
"ConditionStatement", 
"ControlStatement",
"CrewsInvolved",
"Dozers",
"Engines",
"Helicopters",
"Extinguished",
"Fatalities",
"Featured",
"Final",
"FuelType",
"Injuries",
"Location",
# "Name",
"PercentContained",
"Public",
"SearchDescription",
"SearchKeywords",
"Status",
"StructuresDamaged",
"StructuresDestroyed",
"StructuresEvacuated",
"StructuresThreatened",
"UniqueId",
"Updated",
"WaterTenders"
], axis=1)
fire_df.head()

,AcresBurned,ArchiveYear,CalFireIncident,Counties,CountyIds,Latitude,Longitude,MajorIncident,Name,PersonnelInvolved,Started
0,257314.0,2013,True,Tuolumne,55,37.857000,-120.086000,False,Rim Fire,NaN,2013-08-17T15:25:00Z
1,30274.0,2013,True,Los Angeles,19,34.585595,-118.423176,False,Powerhouse Fire,NaN,2013-05-30T15:28:00Z
2,27531.0,2013,True,Riverside,33,33.709500,-116.728850,False,Mountain Fire,NaN,2013-07-15T13:43:00Z
3,27440.0,2013,False,Placer,31,39.120000,-120.650000,False,American Fire,NaN,2013-08-10T16:30:00Z
4,24251.0,2013,True,Ventura,56,0.000000,0.000000,True,Springs Fire,2167.0,2013-05-02T07:01:00Z


In [4]:
# LOOK AT UNDECIDED COUNTIES
[(column, fire_df[column].nunique()) for column in fire_df]

[('AcresBurned', 634),
 ('ArchiveYear', 7),
 ('CalFireIncident', 2),
 ('Counties', 59),
 ('CountyIds', 79),
 ('Latitude', 1441),
 ('Longitude', 1448),
 ('MajorIncident', 2),
 ('Name', 1193),
 ('PersonnelInvolved', 151),
 ('Started', 1597)]

In [5]:
[(column, fire_df[column].unique()) for column in fire_df[["Counties","CountyIds"]]]

[('Counties',
  array(['Tuolumne', 'Los Angeles', 'Riverside', 'Placer', 'Ventura',
         'Fresno', 'Siskiyou', 'Humboldt', 'Tehama', 'Shasta', 'San Diego',
         'Kern', 'Sonoma', 'Contra Costa', 'Butte', 'Tulare',
         'Santa Barbara', 'Mariposa', 'Monterey', 'El Dorado',
         'San Bernardino', 'Plumas', 'Modoc', 'San Luis Obispo', 'Madera',
         'Inyo', 'Napa', 'San Benito', 'San Joaquin', 'Lake', 'Alameda',
         'Glenn', 'Yolo', 'Sacramento', 'Stanislaus', 'Solano', 'Merced',
         'Mendocino', 'Lassen', 'Amador', 'Yuba', 'Nevada', 'Santa Clara',
         'Calaveras', 'San Mateo', 'Orange', 'Colusa', 'Trinity',
         'Del Norte', 'Mono', 'Alpine', 'Sutter', 'Kings', 'Sierra',
         'Santa Cruz', 'Marin', 'Mexico', 'State of Oregon',
         'State of Nevada'], dtype=object)),
 ('CountyIds',
  array(['55', '19', '33', '31', '56', '10', '47', '12', '52', '45', '37',
         '15,56', '49', '7', '4', '54', '42', '22', '33,', '27', '15', '9',
         '3

In [6]:
fire_df[fire_df["CountyIds"] == '6,11,17,23']

,AcresBurned,ArchiveYear,CalFireIncident,Counties,CountyIds,Latitude,Longitude,MajorIncident,Name,PersonnelInvolved,Started
1054,410203.0,2018,True,Colusa,"6,11,17,23",39.243283,-123.103367,True,Ranch Fire (Mendocino Complex),NaN,2018-07-27T12:05:00Z
1055,410203.0,2018,True,Glenn,"6,11,17,23",39.243283,-123.103367,True,Ranch Fire (Mendocino Complex),NaN,2018-07-27T12:05:00Z
1056,410203.0,2018,True,Lake,"6,11,17,23",39.243283,-123.103367,True,Ranch Fire (Mendocino Complex),NaN,2018-07-27T12:05:00Z
1057,410203.0,2018,True,Mendocino,"6,11,17,23",39.243283,-123.103367,True,Ranch Fire (Mendocino Complex),NaN,2018-07-27T12:05:00Z


In [7]:
fire_df[fire_df["CountyIds"] == '6']

,AcresBurned,ArchiveYear,CalFireIncident,Counties,CountyIds,Latitude,Longitude,MajorIncident,Name,PersonnelInvolved,Started
841,70.0,2017,True,Colusa,6,39.012240,-122.201270,True,Sand Fire,76.0,2017-07-17T13:17:00Z
1218,75.0,2018,True,Colusa,6,39.297600,-122.512012,True,Stony Fire,NaN,2018-07-04T13:14:00Z
1599,20.0,2019,True,Colusa,6,39.088333,-122.309167,True,Spring Fire,65.0,2019-08-14T12:52:00Z


In [8]:
# ****** DATASET 2 : THE ENVIRONMENTAL DATA *******

env_df = pd.read_csv("Data/all_env_conditions.csv")

In [9]:
env_df = env_df.drop(["Target", "CIMIS Region"], axis=1)
env_df.head()

,Stn Id,Stn Name,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F)
0,2,FivePoints,1/1/2018,0.06,0.00,219.0,7.3,63.4,35.3,47.8,82.0,46.0,65.0,36.6,3.3,78.3,51.1
1,2,FivePoints,1/2/2018,0.04,0.00,127.0,7.4,59.8,37.7,47.2,80.0,52.0,67.0,36.7,3.1,74.5,51.3
2,2,FivePoints,1/3/2018,0.04,0.00,125.0,8.4,61.1,37.3,49.9,79.0,49.0,68.0,39.9,4.5,107.5,51.3
3,2,FivePoints,1/4/2018,0.07,0.01,219.0,11.6,69.2,48.7,56.8,94.0,52.0,74.0,48.5,5.8,140.2,53.0
4,2,FivePoints,1/5/2018,0.07,0.00,239.0,12.7,73.8,47.5,59.8,94.0,49.0,72.0,50.8,4.2,101.4,54.4


In [10]:
# ADD THE "YEAR" COLUMN TO THE DATABASE
env_df["Year"] = pd.DatetimeIndex(env_df["Date"]).year

In [11]:
env_df.head()

,Stn Id,Stn Name,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F),Year
0,2,FivePoints,1/1/2018,0.06,0.00,219.0,7.3,63.4,35.3,47.8,82.0,46.0,65.0,36.6,3.3,78.3,51.1,2018
1,2,FivePoints,1/2/2018,0.04,0.00,127.0,7.4,59.8,37.7,47.2,80.0,52.0,67.0,36.7,3.1,74.5,51.3,2018
2,2,FivePoints,1/3/2018,0.04,0.00,125.0,8.4,61.1,37.3,49.9,79.0,49.0,68.0,39.9,4.5,107.5,51.3,2018
3,2,FivePoints,1/4/2018,0.07,0.01,219.0,11.6,69.2,48.7,56.8,94.0,52.0,74.0,48.5,5.8,140.2,53.0,2018
4,2,FivePoints,1/5/2018,0.07,0.00,239.0,12.7,73.8,47.5,59.8,94.0,49.0,72.0,50.8,4.2,101.4,54.4,2018


In [12]:
active_stations_df = pd.read_csv("Data/Active_Fire_Stations.csv")
active_stations_df = active_stations_df.drop(["Status", "Name"], axis=1) 

In [13]:
active_stations_df.head()

,Station ID,County
0,2,Fresno
1,5,Kern
2,6,Yolo
3,7,Fresno
4,12,Butte


In [14]:
env_df = env_df.merge(right=active_stations_df, left_on="Stn Id", right_on="Station ID").drop("Station ID", axis=1)

In [15]:
env_df.head()

,Stn Id,Stn Name,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F),Year,County
0,2,FivePoints,1/1/2018,0.06,0.00,219.0,7.3,63.4,35.3,47.8,82.0,46.0,65.0,36.6,3.3,78.3,51.1,2018,Fresno
1,2,FivePoints,1/2/2018,0.04,0.00,127.0,7.4,59.8,37.7,47.2,80.0,52.0,67.0,36.7,3.1,74.5,51.3,2018,Fresno
2,2,FivePoints,1/3/2018,0.04,0.00,125.0,8.4,61.1,37.3,49.9,79.0,49.0,68.0,39.9,4.5,107.5,51.3,2018,Fresno
3,2,FivePoints,1/4/2018,0.07,0.01,219.0,11.6,69.2,48.7,56.8,94.0,52.0,74.0,48.5,5.8,140.2,53.0,2018,Fresno
4,2,FivePoints,1/5/2018,0.07,0.00,239.0,12.7,73.8,47.5,59.8,94.0,49.0,72.0,50.8,4.2,101.4,54.4,2018,Fresno


In [16]:
env_df = env_df.groupby([ "County","Year"]).agg({
    "ETo (in)":"sum",
    "Precip (in)":"sum",
    "Sol Rad (Ly/day)" : "mean",
    "Avg Vap Pres (mBars)": "mean",
    "Max Air Temp (F)":"max",
    "Min Air Temp (F)":"min",
    "Avg Air Temp (F)":"mean",
    "Max Rel Hum (%)":"max",
    "Min Rel Hum (%)":"min",
    "Avg Rel Hum (%)":"mean",
    "Dew Point (F)":"mean",
    "Avg Wind Speed (mph)":"mean",
    "Wind Run (miles)":"mean",
    "Avg Soil Temp (F)":"mean"
    
}).reset_index()
env_df.head()

,County,Year,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Wind Run (miles),Avg Soil Temp (F)
0,Alameda,2018,132.60,40.31,437.525880,11.453520,99.3,19.9,57.542340,100.0,9.0,69.966874,47.291408,3.950828,94.794306,61.806315
1,Alameda,2019,141.87,57.87,430.446849,11.761336,101.6,26.8,57.896237,100.0,9.0,70.883349,47.977046,3.985042,95.649765,60.736218
2,Alameda,2020,116.04,24.83,494.154263,11.959811,108.8,28.3,59.092828,100.0,8.0,69.369418,48.479296,4.134100,99.187415,62.526928
3,Alpine,2019,33.83,5.56,460.730570,5.764767,92.0,4.6,50.388601,99.0,9.0,46.461140,28.563731,7.091192,170.177720,51.319171
4,Alpine,2020,47.24,7.60,494.196000,5.687200,95.5,2.6,51.298000,99.0,7.0,44.064000,28.673200,7.628000,183.086800,50.794800


In [20]:
env_df.to_csv("environment_data_june24.csv")

In [18]:
# # ****** DATASET 3 : THE ELECTRICITY USAGE DATA *******

# electric_df = pd.read_csv("Data/ElectricityByCounty CALIFORNIA Usage in Millions of kWh.csv")

In [19]:
# Find the totals for each sector, and then drop that column.
electric_df = electric_df[electric_df["Sector"] == "Total"].drop(["Sector"], axis=1)

# Drop the years before 2013
electric_df = electric_df.iloc[:, :8]

NameError: name 'electric_df' is not defined

In [ ]:
electric_df.head()

In [ ]:
# # ****** DATASET 4 : THE HOUSING DATA *******

# housing_df = pd.read_csv("Data/housing.csv")

In [ ]:
housing_df = housing_df.drop(["housing_median_age", "total_rooms","total_bedrooms"], axis=1)
housing_df.head()

In [ ]:
# GET THE COUNTY FROM THE LONG/LAT PAIR

In [ ]:
housing_df = housing_df.groupby([ "longitude","latitude"]).agg({
    "population":"sum",
    "households":"sum",
    "median_income" : "mean",
    "median_house_value": "mean",
    "ocean_proximity":"min"
    
}).reset_index()


In [ ]:
housing_df

In [ ]:
housing_df.iloc[[2]]

In [ ]:
# def add_counties(housing_df):
#     counties = []
#     locator = Nominatim(user_agent="Google")
#     i = 0
#     for (long, lat) in housing_df[["longitude", "latitude"]].values: 
#         location = locator.reverse((lat, long))
#         try:
#             counties.append(location.raw["address"]["county"])
#         except:
#             counties.append("NULL")
#         if i % 1000 == 0:
#             pd.DataFrame(counties).to_csv("counties"+str(i)+".csv")
#             counties = []
#         i+=1
#         print(i)
#     housing_df["County"] = counties
    
# add_counties(housing_df)
# # coordinates = 37.88, -122.23
# # location = locator.reverse(coordinates)
# # location.raw

In [ ]:
# counties_df = pd.DataFrame(counties)
# counties_df.to_csv("counties.csv")

In [ ]:
housing_df["County"] = counties_df
housing_df.head()

In [ ]:
# def boxplot_it(df, column_number):
#     print("THERE ARE {} NUMERICAL COLUMNS IN THIS DF AVAILABLE".format( len(df.select_dtypes(include='number').columns)) )
#     number_df = pd.DataFrame(df.select_dtypes(include='number').iloc[:,column_number])
#     [plt.boxplot(x=number_df[column].dropna(), labels=[column]) for column in number_df]

In [ ]:
# boxplot_it(fire_df,4 )
